# Van-der-Pol-Oszillator

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

Der van-der-Pol-Oszillator unterscheidet sich von einem gedämpften harmonischen Oszillator dadurch, dass der zur Geschwindigkeit proportionale Term von der Größe der Auslenkung abhängt. Insbesondere wird der Bewegung bei kleiner Amplitude Energie zugeführt, während die Bewegung bei großer Amplitude gedämpft wird. Dadurch nähert sich die Bewegung für lange Zeiten einem Grenzzyklus an. Konkret lautet die dimensionslose Bewegungsgleichung

$$\ddot x - \varepsilon(1-x^2)\dot x + x = F(t)$$

mit einem positiven Parameter $\varepsilon$.

Im ersten Teil dieses Jupyter-Notebooks werden wir den Fall ohne Anregung, also $F(t)=0$, betrachten, um uns dann im zweiten Teil dem Fall mit harmonischer Anregung $F(t)=F\sin(\omega_\text{ext}t)$ zuzuwenden. Dabei werden wir insbesondere untersuchen, welche Resonanzen sich in der Fouriertransformierten der Amplitude manifestieren. 

### Importanweisungen

Neben den bereits bekannten Importanweisungen wird hier auch das `fft`-Modul aus dem SciPy-Paket importiert, um *Fast Fourier Transform*-Algorithmen zur Berechnung eines Frequenzspektrums zur Verfügung zu haben.

In [ ]:
from math import pi, sin
import numpy as np
from scipy import fft, integrate
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt

plt.style.use("numphyspy.style")

## Van-der-Pol-Oszillator ohne Anregung

### Implementierung der Bewegungsgleichung

Zur numerischen Lösung der Bewegungsgleichung wird die Differentialgleichung 2. Ordnung wieder in zwei Differentialgleichungen 1. Ordnung umgeschrieben.

In [ ]:
def dx_dt(t, x, eps):
    v = x[1]
    a = eps*(1-(x[0]**2))*x[1] - x[0]
    return v, a

### Lösung des Differentialgleichungssystems

Die Lösung der zwei Differentialgleichungen 1. Ordnung erfolgt in der üblichen Weise mit Hilfe der SciPy-Funktion `integrate.solve_ivp`.

In [ ]:
def trajectory(t_end, n_out, eps, x_0, v_0):
    t_values = np.linspace(0, t_end, n_out)
    x_0 = [x_0, v_0]
    solution = integrate.solve_ivp(dx_dt, (0, t_end), x_0,
                                   args=(eps,),
                                   t_eval=t_values)
    x_values, v_values = solution.y[:]
    return t_values, x_values, v_values

### Implementierung der Bedienelemente und graphische Darstellung der zeitabhängigen Amplitude sowie der Bewegung im Phasenraum

Mit Hilfe der Bedienelemente lassen sich die folgenden Parameter einstellen:
- `t_end`: Länge des zu betrachtenden Zeitintervalls
- `n_out`: Anzahl der zu betrachtenden Zeitpunkte
- `eps`: Parameter $\varepsilon$ in der Bewegungsgleichung
- `x_0`: anfängliche Auslenkung
- `v_0`: anfängliche Geschwindigkeit
- `plottype`: Darstellung der Trajektorie über die Zeitabhängigkeit oder im Phasenraum

Je nach Einstellung wird die Bewegung als Zeitabhängigkeit von Amplitude und Geschwindigkeit oder als Phasenraumdarstellung graphisch ausgegeben.

In [ ]:
widget_dict = {"t_end":
               widgets.FloatSlider(
                   value=100, min=10, max=200, step=10,
                   description=r"$t_\text{end}$"),
               "n_out":
               widgets.IntSlider(
                   value=1000, min=1000, max=10000,
                   step=1000,
                   description=r"$n_\text{out}$"),
               "eps":
               widgets.FloatSlider(
                   value=0.1, min=0, max=2, step=0.1,
                   description=r"$\varepsilon$"),
               "x_0":
               widgets.FloatSlider(
                   value=0.1, min=0, max=5, step=0.1,
                   description="$x_0$"),
               "v_0":
               widgets.FloatSlider(
                   value=0, min=-1, max=1, step=0.1,
                   description="$v_0$"),
               "plottype":
               widgets.RadioButtons(
                   options=["Zeitabhängigkeit",
                            "Phasenraum"],
                   description="Darstellung")
               }

@interact(**widget_dict)
def plot_result(t_end, n_out, eps, x_0, v_0, plottype):
    t_values, x_values, v_values = trajectory(t_end, n_out,
                                              eps, x_0, v_0)

    if plottype == "Zeitabhängigkeit":
        fig, (ax1, ax2) = plt.subplots(2, 1)
        ax1.plot(t_values, x_values)
        ax1.set_xlabel("$t$")
        ax1.set_ylabel("$x$")

        ax2.plot(t_values, v_values)
        ax2.set_xlabel("$t$")
        ax2.set_ylabel("$v$")
    else:
        fig, ax = plt.subplots()
        ax.plot(x_values, v_values)
        ax.set_xlabel("$x$")
        ax.set_ylabel("$v$")

### Dynamik eines Ensembles von Phasenraumpunkten

Um die Zeitentwicklung auf einen Grenzzyklus hin zu untersuchen, betrachten wir die Dynamik eines Ensembles, das anfänglich aus  homogen verteilten Zuständen in einem quadratischen Bereich des Phasenraums besteht.

Da die Bewegungsgleichung für jeden Startpunkt individuell gelöst wird, ist hier von einer etwas längeren Rechenzeit auszugehen.

In [ ]:
def trajectories_ensemble(t_end, x_max, v_max, n_max, eps):
    x_init, dx = np.linspace(-x_max, x_max, 2*n_max+1,
                             retstep=True)
    v_init, dv = np.linspace(-v_max, v_max, 2*n_max+1,
                             retstep=True)
    points_x = []
    points_v = []
    for x_0 in x_init:
        for v_0 in v_init:
            solution = integrate.solve_ivp(
                dx_dt, (0, t_end), (x_0, v_0),
                t_eval=[t_end], args=(eps,))
            points_x.append(solution.y[0, 0])
            points_v.append(solution.y[1, 0])
    return points_x, points_v

### Implementierung der Bedienelemente und graphische Darstellung des Phasenraumensembles

Mit den Schiebereglern lassen sich die folgenden Parameter einstellen:
- `t_end`: Zeitpunkt, zu dem das Ensemble dargestellt werden soll
- `x_max`: Betrag der Grenzen für die Amplitude des im Ursprung des Phasenraums zentrierten quadratischen Bereichs
- `v_max`: Betrag der Grenzen für die Geschwindigkeit des im Ursprung des Phasenraums zentrierten quadratischen Bereichs
- `n_max`: bestimmt die Ensemblegröße $(2n_\text{max}+1)^2$
- `eps`: Parameter $\varepsilon$ in der Bewegungsgleichung

Es wird die Position der Phasenraumpunkte zum eingestellten Zeitpunkt $t_\text{end}$ dargestellt.

In [ ]:
widget_dict = {"t_end":
               widgets.FloatSlider(
                   value=1, min=1, max=100, step=1,
                   description=r"$t_\text{end}$"),
               "x_max":
               widgets.FloatSlider(
                   value=7, min=1, max=10, step=1,
                   description=r"$x_\text{max}$"),
               "v_max":
               widgets.FloatSlider(
                   value=7, min=1, max=10, step=1,
                   description=r"$v_\text{max}$"),
               "n_max":
               widgets.IntSlider(
                   value=20, min=10, max=100, step=10,
                   description=r"$n_\text{max}$"),
               "eps":
               widgets.FloatSlider(
                   value=0.1, min=0.1, max=4, step=0.1,
                   description=r"$\varepsilon$"),
               }

interact_start = interact_manual.options(
    manual_name="Start Berechnung")

@interact_start(**widget_dict)
def plot_ensemble(t_end, x_max, v_max, n_max, eps):
    points_x, points_v = trajectories_ensemble(
        t_end, x_max, v_max, n_max, eps)

    fig, ax = plt.subplots()
    ax.scatter(points_x, points_v, s=1)
    ax.set_xlabel("$x$")
    ax.set_ylabel("$v$")

## Van-der-Pol-Oszillator mit harmonischer Anregung

### Implementierung der Bewegungsgleichung

Im Vergleich zur obigen Funktion `dx_dt` wird nun ein Term hinzugefügt, der die harmonische Anregung mit $F(t)=F\sin(\omega_\text{ext}t)$ repräsentiert. 

In [ ]:
def dx_dt_with_excitation(t, x, eps, f, omega):
    v = x[1]
    a = eps*(1-(x[0]**2))*x[1] - x[0] + f*sin(omega*t)
    return v, a

### Lösung des Differentialgleichungssystems und Berechnung des Spektrums

Zunächst wird die Bewegungsgleichung des van-der-Pol-Oszillators mit harmonischer Anregung mit Hilfe der SciPy-Funktion `integrate.solve_ivp` gelöst. Anschließend wird mit der SciPy-Funktion `fft.rfft` eine Fouriertransformation durchgeführt und der Absolutbetrag des komplexwertigen Resultats bestimmt. Die Funktion gibt die Frequenzwerte und das berechnete Spektrum zurück.

In [ ]:
def with_excitation(t_end, n_out, eps, f, omega, x_0, v_0):
    t_values = np.linspace(0, t_end, n_out)
    solution = integrate.solve_ivp(dx_dt_with_excitation,
                                   (0, t_end), (x_0, v_0),
                                   t_eval=t_values,
                                   args=(eps, f, omega))
    x_values, v_values = solution.y[:]
    x_fft = fft.rfft(x_values, norm="forward")
    x_fft_absvalues = np.absolute(x_fft)
    frequency_values = 2*pi/t_end * np.arange(len(x_fft))
    return frequency_values, x_fft_absvalues

### Implementierung der Bedienelemente und graphische Darstellung des Spektrums

Mit Hilfe der Schieberegler lassen sich die folgenden Parameter einstellen:
- `t_end`: zu betrachtendes Zeitintervall
- `log_n_out`: Zweierlogarithmus der Zahl der zu betrachtenden Zeitpunkte
- `eps`: Parameter $\varepsilon$ in der Bewegungsgleichung
- `f`: Anregungsamplitude $F$
- `omega`: dimensionslose Anregungsfrequenz $\omega_\text{ext}$
- `x_0`: anfängliche Amplitude
- `v_0`: anfängliche Geschwindigkeit

Es wird das frequenzabhängige Spektrum in Form des Absolutbetrags der Fouriertransformierten der zeitabhängigen Amplitude dargestellt.

In [ ]:
widget_dict = {"t_end":
               widgets.FloatSlider(
                   value=1000, min=0.1, max=2000, step=0.1,
                   description=r"$t_\text{end}$"),
               "log_n_out":
               widgets.IntSlider(
                   value=12, min=11, max=18,
                   description=r"$\log_2(n_\text{out})$"),
               "eps":
               widgets.FloatSlider(
                   value=0.1, min=0, max=2, step=0.1,
                   description=r"$\varepsilon$"),
               "f":
               widgets.FloatSlider(
                   value=2, min=0, max=4, step=0.1,
                   description="$F$"),
               "omega":
               widgets.FloatSlider(
                   value=1.57, min=0, max=2, step=0.01,
                   description=r"$\omega_\text{ext}$"),
               "x_0":
               widgets.FloatSlider(
                   value=0.1, min=0, max=5, step=0.1,
                   description="$x_0$"),
               "v_0":
               widgets.FloatSlider(
                   value=0, min=-1, max=1, step=0.1,
                   description="$v_0$")
               }

interact_start = interact_manual.options(
    manual_name="Start Berechnung")

@interact_start(**widget_dict)
def plot_spectrum(t_end, log_n_out, eps, f, omega, x_0, v_0):
    n_out = 2**log_n_out
    frequency_values, x_fft_values = with_excitation(
        t_end, n_out, eps, f, omega, x_0, v_0)

    fig, ax = plt.subplots()
    ax.semilogy(frequency_values, x_fft_values)
    ax.set_xlim(0, 6)
    ax.set_xlabel(r"$\omega$")
    ax.set_ylabel("$x$")